In [1]:
import os
import numpy as np
import pandas as pd

pd.set_option("display.max_rows", 5)

In [2]:
dir = os.getcwd()
data_directory = os.path.join(dir,"output")
stattion_list_path = os.path.join(dir,"station list.csv")
#SPECS TO READ DATA OF STATIONS
data_cols = ["year","month","day","v4","tmax","tmin","pre","v8","v9"]
data_specs = [(0,6),(6,10),(10,14),(14,20),(20,26),(26,32),(32,38),(38,42),(42,48)]
data_format = '%6s%4s%4s%6s%6s%6s%6s%6s%6s'



In [9]:

#SPECS TO WRITE SUMMARY DATA
summary_format = '%10s%10.4f%10.4f%10.4f%10.2f%10.2f%10.2f%10.4f'
summary_cols = ["year","mean_tmin","mean_tmax","mean_pre", "sum_pre", "min_tmin", "max_tmax","std_pre"]
summary_path = os.path.join(dir,"summary")

In [7]:
all_stations = [name for name in os.listdir(data_directory) if os.path.isdir(os.path.join(data_directory,name))]
#GET ALL THE STATION INFORMATION FROM LIST
#station_list = pd.read_csv(stattion_list_path, sep=";" )
#FORMAT THE STATION LIST
#station_list['index_no'] = station_list['index_no'].map(lambda x: '%04d'%x)
#STATION_INFO NOW CONTAINS ONLY STATIONS THAT WE HAVE DATA
#station_info = station_list[station_list.index_no.isin(all_stations)]

#pd.set_option("display.max_rows", 2)
#station_info.sort_values(by="district")

In [4]:
def get_stats_for_station(station,dir = dir): #output is the dataframe of average temp and sum pre according to years
    #read the folder
    year = []
    mean_tmin = []
    mean_tmax = []
    mean_pre = []
    max_tmax = []
    min_tmin = []
    std_pre = []
    sum_pre = []
    #READ STATION PATH
    station_path = os.path.join(dir,"output",'%04d'%station)
    #GET ALL STATIONS
    all_datas = [name for name in os.listdir(station_path)]
    #for every file
    for data_name in all_datas:
        #GET DATA PATH OF A YEAR
        data_path = os.path.join(station_path, data_name)
        #GET THE YEAR
        this_year = data_name[4:6] + data_name[7:]
        #READ THE DATA FILE
        data = pd.read_fwf(data_path,skiprows=0,skipfooter=1,colspecs=data_specs,names=data_cols)
        #CALCULATE MEAN AND MAX OF TMAX
        this_mean_tmax = data[data.tmax != 'DNA'].tmax.astype(float).mean()
        this_max_tmax = data[data.tmax != 'DNA'].tmax.astype(float).max()
        #CALCULATE MEAN AND MIN OF TMIN
        this_mean_tmin = data[data.tmin != 'DNA'].tmin.astype(float).mean()
        this_min_tmin = data[data.tmin != 'DNA'].tmin.astype(float).min()
        #CALCULATE MEAN AND STD PRE
        this_mean_pre = data[(data.pre != 'T') & (data.pre !='DNA')].pre.astype('float').mean()
        this_std_pre = data[(data.pre != 'T') & (data.pre !='DNA')].pre.astype('float').std()
        this_sum_pre = data[(data.pre != 'T') & (data.pre !='DNA')].pre.astype('float').sum()
        #append
        year.append(this_year)
        mean_tmin.append(this_mean_tmin)
        mean_tmax.append(this_mean_tmax)
        mean_pre.append(this_mean_pre)
        min_tmin.append(this_min_tmin)
        max_tmax.append(this_max_tmax)
        std_pre.append(this_std_pre)
        sum_pre.append(this_sum_pre)
    output = pd.DataFrame({'year': year,'mean_tmin': mean_tmin, 'mean_tmax': mean_tmax,\
                           'mean_pre': mean_pre, 'sum_pre': sum_pre, 'min_tmin':min_tmin,\
                           'max_tmax': max_tmax, 'std_pre': std_pre})
    return output

In [ ]:
#station_path = os.path.join(dir,"output",'%04d'%104)
#x = [name for name in os.listdir(station_path)]
#a = x[0]
#data_path = os.path.join(station_path, a)
#data_point = pd.read_fwf(data_path,skiprows=0,skipfooter=1,colspecs=data_specs,names=data_cols)
#data_point

In [5]:
def write_calculated_table(data, station):
    np.savetxt(os.path.join(summary_path, str(station)), data.values, fmt = summary_format,\
           header = ','.join(summary_cols)) 

In [11]:
get_stats_for_station(int(104))
#'%10d%10.4f%10.4f%10.4f%10.2f%10.2f%10.2f%10.4f'

,year,mean_tmin,mean_tmax,mean_pre,sum_pre,min_tmin,max_tmax,std_pre
0,1978,12.558437,20.289720,3.921221,1348.9,-0.5,30.2,8.594163
1,1979,12.057618,19.935734,3.436932,1209.8,-1.0,28.7,10.308321
...,...,...,...,...,...,...,...,...
34,2012,11.199451,22.365205,4.183838,1242.6,-2.4,34.3,11.930596
35,2013,11.456044,22.373901,4.587798,1541.5,0.0,31.2,14.434418


In [12]:
for station in all_stations:
    summary_data = get_stats_for_station(int(station))
    write_calculated_table(summary_data, station)